## Analyze the data MB, JP2, HT2

In [ ]:
from IPython.display import display, Math

equation = (r"""
        N_{\text{corresponding events}}^{\text{MB}} = \sum_{\text{runs}} N_{\text{events}}^{\text{JP2}} 
        \times \textcolor{red}{\frac{\text{PS}^{\text{MB}}}{\text{PS}^{\text{JP2}}} }
        \times \textcolor{green}{\frac{\text{LT}^{\text{JP2}}}{\text{LT}^{\text{MB}}} }
        \times \textcolor{orange}{\frac{N_{\text{events,max}}^{\text{MB}}}{N_{\text{events,max}}^{\text{JP2}}}} 
        \times \textcolor{olive}{\frac{R_{\text{MB}}}{R_{\text{JP2}}}}
        \times \text{triggerMatch}
""")
description = r"""
- $ N_{\text{events}}^{\text{JP2}}$ - number of currently analyzed JP2 events  
- $ \textcolor{red}{\text{PS}^{\text{JP2}}(run)}$ - prescale JP2 $=1$  
- $ \textcolor{red}{\text{PS}^{\text{MB}}(run)}$ - prescale minbias $\approx 100$  
- $ \textcolor{green}{\text{LT}^{\text{JP2}}(run)}$ - livetime JP2 $\approx 0.98$  
- $ \textcolor{green}{\text{LT}^{\text{MB}}(run)}$ - livetime minbias $\approx 0.98$  
- $ \textcolor{orange}{N_{\text{events,max}}^{\text{MB}}(run)}$ - maximum number of events MB  
- $ \textcolor{orange}{N_{\text{events,max}}^{\text{JP2}}(run)}$ - maximum number of events JP2  
- $ \textcolor{olive}{R_{\text{MB}}}$ - ratio of passed events MB after vertex cut $\approx 0.8$ 
- $ \textcolor{olive}{R_{\text{JP2}}}$ - ratio of passed events JP2 $\approx 0.6$ 
"""

display(Math(equation))


display(Math(description))


<IPython.core.display.Math object>

<IPython.core.display.Math object>

: 

In [1]:
import ROOT
import uproot
import awkward as ak
import numpy as np
path='/home/prozorov/dev/star/jets_pp_2012/output/'
triggers = ['MB', 'JP2', 'HT2']
tree_name = 'ResultTree'  # Update this to your tree name
branch_names = ['runid1', 'pt', 'ptLead', 'trigger_match_JP2', 'trigger_match_HT2']
pt_vs_run = {}

run_map = {}
run_map_inverse = {}
with open('/home/prozorov/dev/star/jets_pp_2012/macros/ana/run_map.txt', 'r') as f:
    for line in f:
        run, i = line.strip().split()
        run_map[int(run)] = int(i)
        run_map_inverse[int(i)] = int(run)

pt_reco_bins = (
    5.0,  6.0,  7.0,  8.0,  9.0,  10.0, 11.0, 12.0, 13.0, 14.0,
    15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0,
    25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0,
    35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 42.0, 44.0, 46.0, 48.0,
    50.0, 52.0, 54.0, 56.0, 58.0, 60.0, 64.0, 70.0, 90.0)
pt_reco_bins = np.array(pt_reco_bins, dtype='float64')

# pt_reco_bins = np.array(list(range(100)), dtype='float64')
pt_vs_run = {}

for trigger in triggers:
    print(f'Processing trigger {trigger}')
    file_name = path + f'jets_{trigger}.root'
    # if trigger == 'MB':
    #     file_name = path + f'jets_MB_charged.root'
    my_file=ROOT.TFile(file_name)
    analyzed_event_hist= my_file.Get('hEventsRun')
    nbins=analyzed_event_hist.GetNbinsX()
    root_file=uproot.open(file_name)
    tree = root_file[tree_name]
    tree_arrays = tree.arrays(branch_names)
    # convert runid1 to run_mapped
    tree_arrays['run_mapped'] = [run_map[runid] for runid in tree_arrays['runid1']]
    # add trigger_match_MB which is always 1
    if trigger == 'MB':
        tree_arrays['trigger_match_MB'] = np.ones(len(tree_arrays['pt']), dtype='int32')
    df = ak.to_rdataframe({"pt": tree_arrays['pt'], "ptLead": tree_arrays['ptLead'],
                                "run_mapped": tree_arrays['run_mapped'], "trigger_match": tree_arrays[f'trigger_match_{trigger}']})
    pt_vs_run[trigger] = df.Histo2D((f'{trigger}_pt_vs_run', f'{trigger} pt; run ;p_t, GeV/c; counts',nbins,0,nbins, len(pt_reco_bins)-1, pt_reco_bins), "run_mapped","pt")
    # set labels equal to runid
    for i in range(1, nbins+1):
        pt_vs_run[trigger].GetXaxis().SetBinLabel(i, str(lumi.index[i-1]))


outfile_2d = ROOT.TFile("pt_vs_run.root", "RECREATE")
for trigger in triggers:
    pt_vs_run[trigger].Write()
outfile_2d.Close()

Error in <TExMap::Add>: key 140040869539408 is not unique
Error in <TExMap::Add>: key 140040870121472 is not unique
TClassTable::Add:0: RuntimeWarning: class timespec already in TClassTable
TClassTable::Add:0: RuntimeWarning: class reverse_iterator<__gnu_cxx::__normal_iterator<TString*,vector<TString> > > already in TClassTable
TClassTable::Add:0: RuntimeWarning: class reverse_iterator<__gnu_cxx::__normal_iterator<int*,vector<int> > > already in TClassTable
TClassTable::Add:0: RuntimeWarning: class reverse_iterator<__gnu_cxx::__normal_iterator<string*,vector<string> > > already in TClassTable
TClassTable::Add:0: RuntimeWarning: class __gnu_cxx::__normal_iterator<TString*,vector<TString> > already in TClassTable
TClassTable::Add:0: RuntimeWarning: class __gnu_cxx::__normal_iterator<const TString*,vector<TString> > already in TClassTable
TClassTable::Add:0: RuntimeWarning: class __gnu_cxx::__normal_iterator<char*,string> already in TClassTable
TClassTable::Add:0: RuntimeWarning: class __

Processing trigger MB


Warning in <TClass::Init>: no dictionary for class TStarJetVectorJet is available
Warning in <TClass::Init>: no dictionary for class TStarJetVector is available
Warning in <TColor::TColor>: color 2000 already defined
Warning in <TColor::TColor>: color 2001 already defined
Warning in <TColor::TColor>: color 2002 already defined
Warning in <TColor::TColor>: color 2003 already defined
Warning in <TColor::TColor>: color 2004 already defined
Warning in <TColor::TColor>: color 2005 already defined
Warning in <TColor::TColor>: color 2006 already defined
Warning in <TColor::TColor>: color 2007 already defined
Warning in <TColor::TColor>: color 2008 already defined
Warning in <TColor::TColor>: color 2009 already defined
Warning in <TColor::TColor>: color 2010 already defined
Warning in <TColor::TColor>: color 2011 already defined
Warning in <TColor::TColor>: color 2012 already defined
Warning in <TColor::TColor>: color 3000 already defined
Warning in <TColor::TColor>: color 3001 already defined

NameError: name 'lumi' is not defined

In [ ]:
pt_vs_run_raw = {}
pt_vs_run_trigger = {}

infile_2d = ROOT.TFile("pt_vs_run.root", "READ")
for trigger in triggers:
    pt_vs_run_raw[trigger] = infile_2d.Get(f'{trigger}_pt_vs_run')
    pt_vs_run_raw[trigger].SetDirectory(0)
    if trigger in ['JP2', 'HT2']:
        pt_vs_run_trigger[trigger] = infile_2d.Get(f'{trigger}_pt_vs_run_trigger')
        pt_vs_run_trigger[trigger].SetDirectory(0)

pt_vs_run_trigger['MB']= pt_vs_run_raw['MB']




analyzed_events = {}
for trigger in triggers:
    file_name = path + f'jets_{trigger}.root'
    my_file=ROOT.TFile(file_name)
    analyzed_event_hist= my_file.Get('hEventsRun').Clone(f'hEventsRun_{trigger}')
    analyzed_event_hist.SetTitle(f'Analyzed events {trigger};runid;counts')
    analyzed_event_hist.SetDirectory(0)
    analyzed_events[trigger] = analyzed_event_hist

In [ ]:
can=ROOT.TCanvas('can','can',800,600)
can.Divide(2,3)
can.Draw()

can.cd(1)
pt_vs_run_raw['MB'].Draw('colz')

can.cd(3)
pt_vs_run_raw['JP2'].Draw('colz')

can.cd(4)
pt_vs_run_trigger['JP2'].Draw('colz')

can.cd(5)
pt_vs_run_raw['HT2'].Draw('colz')

can.cd(6)
pt_vs_run_trigger['HT2'].Draw('colz')


In [ ]:
can=ROOT.TCanvas('can_events','',1600,600)
can.Divide(3,2)
can.Draw()

can.cd(1)
analyzed_events['MB'].Draw('')

can.cd(2)
analyzed_events['JP2'].Draw('')

can.cd(3)
analyzed_events['HT2'].Draw('')

ratio_analyzed_events={}

can.cd(5)
ratio_analyzed_events['JP2'] = analyzed_events['JP2'].Clone('ratio_JP2')
ratio_analyzed_events['JP2'].Divide(analyzed_events['MB'])
ratio_analyzed_events['JP2'].Draw('')
ratio_analyzed_events['JP2'].SetTitle('JP2/MB')

can.cd(6)
ratio_analyzed_events['HT2'] = analyzed_events['HT2'].Clone('ratio_HT2')
ratio_analyzed_events['HT2'].Divide(analyzed_events['MB'])
ratio_analyzed_events['HT2'].Draw('')
ratio_analyzed_events['HT2'].SetTitle('HT2/MB')



In [ ]:
# load histograms
import ROOT
pt_vs_run_raw = {}
pt_vs_run_trigger = {}
infile_2d = ROOT.TFile("pt_vs_run.root")
triggers = ['MB', 'JP2', 'HT2']
for trigger in triggers:
    pt_vs_run_raw[trigger] = infile_2d.Get(f'{trigger}_pt_vs_run')
    if trigger in ['JP2', 'HT2']:
        pt_vs_run_trigger[trigger] = infile_2d.Get(f'{trigger}_pt_vs_run_trigger')
 
bad_runs_JP2=[  13048019,13048092,13048093,13049006,13049007,13051074,13051099,13052061,13061035,13064067,13068060,13069023,13070061,13063012]
# make a  function to multiply 2D histogram x:runid vs y:pt by 1D histogram x:runid

def divide_by_run(pt_vs_run, run_hist, name, ratio_hist=None):
    pt_vs_run_copy = pt_vs_run.Clone(name)

    for i in range(1, pt_vs_run.GetNbinsX() + 1):
        bin_content = run_hist.GetBinContent(i)
        runid= run_hist.GetXaxis().GetBinLabel(i)
        # consider only those runs which has ratio_hist bin content > 0
        if ratio_hist:
            if ratio_hist.GetBinContent(i) == 0:
                bin_content = 0
        # exclude bad runs
        if int(runid) in bad_runs_JP2:
            bin_content=0
        if bin_content == 0:
            scale = 0
        else:
            scale = 1 / bin_content

        for j in range(1, pt_vs_run.GetNbinsY() + 1):
            pt_vs_run_copy.SetBinContent(i, j, pt_vs_run.GetBinContent(i, j) * scale)
            pt_vs_run_copy.SetBinError(i, j, pt_vs_run.GetBinError(i, j) * scale)
    return pt_vs_run_copy


def multiply_by_run(pt_vs_run, run_hist, name):
    pt_vs_run_copy = pt_vs_run.Clone(name)

    for i in range(1, pt_vs_run.GetNbinsX() + 1):
        bin_content = run_hist.GetBinContent(i)
        runid= run_hist.GetXaxis().GetBinLabel(i)
        # exclude bad runs
        if int(runid) in bad_runs_JP2:
            bin_content=0
    
        scale = bin_content

        for j in range(1, pt_vs_run.GetNbinsY() + 1):
            pt_vs_run_copy.SetBinContent(i, j, pt_vs_run.GetBinContent(i, j) * scale)
            pt_vs_run_copy.SetBinError(i, j, pt_vs_run.GetBinError(i, j) * scale)
    return pt_vs_run_copy 


lumi=ROOT.TFile("lumi.root")

    # prescale_ratio =  f'prescale_MB_over_{trigger}'
    # livetime_ratio =  f'livetime_{trigger}_over_MB'
    # maxnevent_ratio =  f'nevents_MB_over_{trigger}'

# divide by analyzed events
normalized_events = {}

# for trigger in triggers:
#     normalized_events[trigger] = divide_by_run(pt_vs_run_raw[trigger], analyzed_events[trigger], f'{trigger}_normalized')
    
normalized_triggered_events = {}
normalized_events_livetime = {}
normalized_events_livetime_prescale = {}
normalized_events_livetime_prescale_maxnevents = {}
total_cross_section = {}
normalized_mb_events = {}

for trigger in ['JP2', 'HT2']:

    normalized_mb_events[trigger] = divide_by_run(pt_vs_run_raw['MB'],analyzed_events['MB'], f'MB_for_{trigger}', ratio_hist=ratio_analyzed_events[trigger])

    normalized_events[trigger] = divide_by_run(pt_vs_run_raw[trigger], analyzed_events[trigger], f'{trigger}_normalized')

    normalized_triggered_events[trigger] = divide_by_run(pt_vs_run_trigger[trigger], analyzed_events[trigger], f'{trigger}_triggered_normalized')
# divide by livetime factor
    livetime_ratio= lumi.Get(f'livetime_{trigger}_over_MB')
    normalized_events_livetime[trigger] = divide_by_run(normalized_triggered_events[trigger], livetime_ratio, f'{trigger}_livetime')
# divide by prescale factor
    prescale_ratio= lumi.Get(f'prescale_MB_over_{trigger}')
    normalized_events_livetime_prescale[trigger] = divide_by_run(normalized_events_livetime[trigger], prescale_ratio, f'{trigger}_prescale')
# divide by maxnevents factor
    maxnevent_ratio= lumi.Get(f'nevents_MB_over_{trigger}')
    normalized_events_livetime_prescale_maxnevents[trigger] =divide_by_run(normalized_events_livetime_prescale[trigger], maxnevent_ratio, f'{trigger}_maxnevents')
    #divide by luminosity
    luminosity = lumi.Get(f'luminosity_{trigger}')
    total_cross_section[trigger] = divide_by_run(normalized_events_livetime_prescale_maxnevents[trigger], luminosity, f'{trigger}_cross_section')




# make 1d histogram out of this histograms
def get_1d(pt_vs_run, name):
    analyzed_runs=0
    temp = pt_vs_run.ProjectionY('', 1, 1)
    output=ROOT.TH1D(name+'_per_run', name+ ';'+pt_vs_run.GetYaxis().GetTitle()+';counts per event', temp.GetNbinsX(), temp.GetXaxis().GetXmin(), temp.GetXaxis().GetXmax())
    for i in range(1, pt_vs_run.GetNbinsX() + 1):
         projection = pt_vs_run.ProjectionY(f'{name}_{i}', i, i)
         if projection.Integral() == 0:
            continue
         analyzed_runs+=1
         output.Add(projection)
    output.Scale(1/analyzed_runs)
    
    return output


normalized_mb_events_1d = {}
normalized_events_1d = {}
normalized_triggered_events_1d = {}
normalized_events_livetime_1d = {}
normalized_events_livetime_prescale_1d = {}
normalized_events_livetime_prescale_maxnevents_1d = {}
total_cross_section_1d = {}

# save histograms
outfile_2d = ROOT.TFile("pt_vs_run_normalized.root", "RECREATE")
for trigger in ['JP2', 'HT2']:
    normalized_mb_events[trigger].Write()
    normalized_mb_events_1d[trigger] = get_1d(normalized_mb_events[trigger], 'MB_for'+trigger)
    normalized_mb_events_1d[trigger].Write()

    normalized_events[trigger].Write()
    normalized_events_1d[trigger] = get_1d(normalized_events[trigger], trigger)
    normalized_events_1d[trigger].Write()

    normalized_triggered_events[trigger].Write()
    normalized_events_livetime[trigger].Write()
    normalized_events_livetime_prescale[trigger].Write()
    normalized_events_livetime_prescale_maxnevents[trigger].Write()

    normalized_triggered_events_1d[trigger] = get_1d(normalized_triggered_events[trigger], trigger+'_trigger')
    normalized_triggered_events_1d[trigger].Write()
    normalized_events_livetime_1d[trigger] = get_1d(normalized_events_livetime[trigger], trigger+'_livetime')
    normalized_events_livetime_1d[trigger].Write()
    normalized_events_livetime_prescale_1d[trigger] = get_1d(normalized_events_livetime_prescale[trigger], trigger+'_prescale')
    normalized_events_livetime_prescale_1d[trigger].Write()
    normalized_events_livetime_prescale_maxnevents_1d[trigger] = get_1d(normalized_events_livetime_prescale_maxnevents[trigger], trigger+'_maxnevents')
    normalized_events_livetime_prescale_maxnevents_1d[trigger].Write()

    total_cross_section[trigger].Write()
    total_cross_section_1d[trigger] = get_1d(total_cross_section[trigger], f'cross_section_{trigger}')
    total_cross_section_1d[trigger].Write()

# outfile_2d.Close()

In [ ]:
# go by bin by bin and multiply by trigger efficiency interpolated
def divide_by_efficiency(data, eff):
    
    for i in range(1, data.GetNbinsX()+1):
        bin_content = data.GetBinContent(i)
        bin_error = data.GetBinError(i)
        bin_center = data.GetBinCenter(i)
        # get trigger efficiency
        eff_value = eff.GetBinContent(eff.GetXaxis().FindBin(bin_center))
        eff_error = eff.GetBinError(eff.GetXaxis().FindBin(bin_center))
        # check if trigger efficiency is 0
        if eff_value == 0:
            eff_value = 1
        # set new value
        data.SetBinContent(i, bin_content /eff_value)
        # calculate uncertainty propagation
        # check if not 0
        if bin_content == 0:
            continue
        new_bin_error = bin_content /eff_value * math.sqrt((bin_error/bin_content)**2 + (eff_error/eff_value)**2)
        data.SetBinError(i, new_bin_error)

def multiply_by_efficiency(data, eff):
    for i in range(1, data.GetNbinsX()+1):
        bin_content = data.GetBinContent(i)
        bin_error = data.GetBinError(i)
        bin_center = data.GetBinCenter(i)
        # get trigger efficiency
        eff_value = eff.GetBinContent(eff.GetXaxis().FindBin(bin_center))
        eff_error = eff.GetBinError(eff.GetXaxis().FindBin(bin_center))
        # check if trigger efficiency is 0
        if eff_value == 0:
            eff_value = 1
        # set new value
        data.SetBinContent(i, bin_content *eff_value)
        # calculate uncertainty propagation
        # check if not 0
        if bin_content == 0:
            continue
        new_bin_error = bin_content *eff_value * math.sqrt((bin_error/bin_content)**2 + (eff_error/eff_value)**2)
        data.SetBinError(i, new_bin_error)

In [ ]:
# plot all 1# compare 1D distributions
can1d = ROOT.TCanvas('can1d', 'can1d', 800, 600)
title = ';p_{t}, GeV/c; counts per event'

trigger='HT2'
# trigger='JP2'
import math

input_file=  ROOT.TFile(f"./plots/embedding_root/matched_over_all_reco.root", "READ")
matching_efficiency = input_file.Get(f"h_ratio").Clone(f'matching_efficiency')

trigger_efficiency={}
trigger_mc_efficiency={}

# load trigger efficiency from embedding
for trig in ['JP2', 'HT2']:
    input_file = ROOT.TFile(f"./plots/embedding_root/{trig}_over_matched_reco.root", "READ")
    trigger_efficiency[trig]=input_file.Get(f"h_ratio").Clone(f'h_ratio_{trig}')
    input_file_mc = ROOT.TFile(f"./plots/embedding_root/{trig}_over_matched_mc.root", "READ")
    trigger_mc_efficiency[trig]=input_file_mc.Get(f"h_ratio"). Clone(f'h_ratio_mc_{trig}')
    trigger_efficiency[trig].SetDirectory(0)
    trigger_mc_efficiency[trig].SetDirectory(0)

mb_norm = normalized_mb_events_1d[trigger]

h1_norm_trigger                       = normalized_triggered_events_1d[trigger]
h1_norm_trigger_lt                    = normalized_events_livetime_1d[trigger]
h1_norm_trigger_lt_ps                 = normalized_events_livetime_prescale_1d[trigger]
h1_norm_trigger_lt_ps_maxevt          = normalized_events_livetime_prescale_maxnevents_1d[trigger]

# clone histogram for trigger efficiency
h1_final = h1_norm_trigger_lt_ps_maxevt.Clone(f'{trigger}_trigger_efficiency')
h1_final.SetTitle(f'{trigger} trigger efficiency')

# go by bin by bin and multiply by trigger efficiency interpolated
def divide_by_efficiency(data, eff):
    
    for i in range(1, data.GetNbinsX()+1):
        bin_content = data.GetBinContent(i)
        bin_error = data.GetBinError(i)
        bin_center = data.GetBinCenter(i)
        # get trigger efficiency
        eff_value = eff.GetBinContent(eff.GetXaxis().FindBin(bin_center))
        eff_error = eff.GetBinError(eff.GetXaxis().FindBin(bin_center))
        # check if trigger efficiency is 0
        if eff_value == 0:
            eff_value = 1
        # set new value
        data.SetBinContent(i, bin_content /eff_value)
        # calculate uncertainty propagation
        # check if not 0
        if bin_content == 0:
            continue
        new_bin_error = bin_content /eff_value * math.sqrt((bin_error/bin_content)**2 + (eff_error/eff_value)**2)
        data.SetBinError(i, new_bin_error)

def multiply_by_efficiency(data, eff):
    for i in range(1, data.GetNbinsX()+1):
        bin_content = data.GetBinContent(i)
        bin_error = data.GetBinError(i)
        bin_center = data.GetBinCenter(i)
        # get trigger efficiency
        eff_value = eff.GetBinContent(eff.GetXaxis().FindBin(bin_center))
        eff_error = eff.GetBinError(eff.GetXaxis().FindBin(bin_center))
        # check if trigger efficiency is 0
        if eff_value == 0:
            eff_value = 1
        # set new value
        data.SetBinContent(i, bin_content *eff_value)
        # calculate uncertainty propagation
        # check if not 0
        if bin_content == 0:
            continue
        new_bin_error = bin_content *eff_value * math.sqrt((bin_error/bin_content)**2 + (eff_error/eff_value)**2)
        data.SetBinError(i, new_bin_error)

divide_by_efficiency(h1_final, trigger_efficiency[trigger])
# divide_by_efficiency(mb_norm, trigger_efficiency["MB"])


# scale by ratio_passed_events
h1_final.Scale(ratio_passed_events[trigger])
# h1_final.Scale(1./2) # non physical - missing factor 
can1d.Draw()
can1d.SetLogy()

h1_norm_trigger.SetMarkerColor(ROOT.kBlack)
h1_norm_trigger.SetLineColor(ROOT.kBlack)
h1_norm_trigger.SetMarkerStyle (24)
h1_norm_trigger.SetMarkerSize(0.8)

h1_norm_trigger_lt_ps.SetMarkerColor(ROOT.kRed) 
h1_norm_trigger_lt_ps.SetLineColor(ROOT.kRed)
h1_norm_trigger_lt_ps.SetMarkerStyle (20)
h1_norm_trigger_lt_ps.SetMarkerSize(0.8)
h1_norm_trigger_lt.SetMarkerColor(ROOT.kMagenta)
h1_norm_trigger_lt.SetLineColor(ROOT.kMagenta)
h1_norm_trigger_lt.SetMarkerStyle (21)
h1_norm_trigger_lt.SetMarkerSize(0.8)
h1_norm_trigger_lt_ps_maxevt.SetMarkerColor(ROOT.kBlue-1)
h1_norm_trigger_lt_ps_maxevt.SetLineColor(ROOT.kBlue-1)
h1_norm_trigger_lt_ps_maxevt.SetMarkerStyle (22)
h1_norm_trigger_lt_ps_maxevt.SetMarkerSize(0.8)

h1_final.SetMarkerColor(ROOT.kGreen+2)
h1_final.SetLineColor(ROOT.kGreen+2)
h1_final.SetMarkerStyle (23)
h1_final.SetMarkerSize(0.8)



mb_norm.SetMarkerColor(ROOT.kAzure)
mb_norm.SetLineColor(ROOT.kAzure)
mb_norm.SetMarkerStyle (24)
mb_norm.SetMarkerSize(0.8)
h1_norm_trigger.GetYaxis().SetRangeUser(1e-9, h1_norm_trigger.GetMaximum()*1.05)
h1_norm_trigger.GetXaxis().SetRangeUser(5, 40)
h1_norm_trigger.Draw('pe')
h1_norm_trigger_lt.Draw('same pe')
h1_norm_trigger_lt_ps.Draw('same pe')
h1_norm_trigger_lt_ps_maxevt.Draw('same pe')
h1_final.Draw('same pe')
mb_norm.Draw('same pe')
can1d.BuildLegend()
can1d.Draw()

can1d.SaveAs(f'plots/{trigger}_1d.pdf')



Info in <TCanvas::Print>: pdf file plots/HT2_1d.pdf has been created


In [ ]:
canvas.SaveAs ("plots/ratio_MB_JP2_jet_yields.pdf")

Info in <TCanvas::Print>: pdf file plots/ratio_MB_JP2_jet_yields.pdf has been created


In [ ]:
import math
# go by bin by bin and multiply by trigger efficiency interpolated
def divide_by_efficiency(data, eff):
    new_hist = data.Clone(f'{data.GetName()}_new')
    
    for i in range(1, data.GetNbinsX()+1):
        bin_content = data.GetBinContent(i)
        bin_error = data.GetBinError(i)
        bin_center = data.GetBinCenter(i)
        # get trigger efficiency
        eff_value = eff.GetBinContent(eff.GetXaxis().FindBin(bin_center))
        eff_error = eff.GetBinError(eff.GetXaxis().FindBin(bin_center))
        # check if trigger efficiency is 0
        if eff_value == 0:
            eff_value = 1
        # set new value
        new_hist.SetBinContent(i, bin_content /eff_value)
        # calculate uncertainty propagation
        # check if not 0
        if bin_content == 0:
            continue
        new_bin_error = bin_content /eff_value * math.sqrt((bin_error/bin_content)**2 + (eff_error/eff_value)**2)
        new_hist.SetBinError(i, new_bin_error)
    return new_hist

# def multiply_by_efficiency(data, eff):
#     for i in range(1, data.GetNbinsX()+1):
#         bin_content = data.GetBinContent(i)
#         bin_error = data.GetBinError(i)
#         bin_center = data.GetBinCenter(i)
#         # get trigger efficiency
#         eff_value = eff.GetBinContent(eff.GetXaxis().FindBin(bin_center))
#         eff_error = eff.GetBinError(eff.GetXaxis().FindBin(bin_center))
#         # check if trigger efficiency is 0
#         if eff_value == 0:
#             eff_value = 1
#         # set new value
#         data.SetBinContent(i, bin_content *eff_value)
#         # calculate uncertainty propagation
#         # check if not 0
#         if bin_content == 0:
#             continue
#         new_bin_error = bin_content *eff_value * math.sqrt((bin_error/bin_content)**2 + (eff_error/eff_value)**2)
#         data.SetBinError(i, new_bin_error)

In [ ]:
# compare all triggers

canvas=ROOT.TCanvas('can2','',800,600)
canvas.Draw()
colors = {
    'MB' : ROOT.kBlue+1,
    'JP2': ROOT.kViolet-4,
    'HT2': ROOT.kGreen-3,
}

markers = {
    'MB' : 20,
    'JP2': 21,
    'HT2': 22,
}


mb_norm = normalized_mb_events_1d['JP2']
mb_norm.SetLineColor(colors['MB'])
mb_norm.SetMarkerColor( colors['MB'])
mb_norm.SetMarkerStyle( markers['MB'])
mb_norm.SetMarkerSize(1.2)

# input_file=  ROOT.TFile(f"./plots/embedding_root/matched_over_all_reco.root", "READ")
# matching_efficiency = input_file.Get(f"h_ratio").Clone(f'matching_efficiency')
trigger_efficiency={}
trigger_mc_efficiency={}
for trig in ['JP2', 'HT2']:
    # input_file = ROOT.TFile(f"./plots/embedding_root/{trig}_reco.root", "READ")
    input_file = ROOT.TFile(f"./plots/embedding_root/{trig}_over_matched_reco.root", "READ")
    trigger_efficiency[trig]=input_file.Get(f"h_ratio").Clone(f'trigger_efficiency_{trig}')
    trigger_efficiency[trig].SetDirectory(0)
    input_file_mc = ROOT.TFile(f"./plots/embedding_root/{trig}_over_matched_mc.root", "READ")
    trigger_mc_efficiency[trig]=input_file_mc.Get(f"h_ratio"). Clone(f'trigger_mc_efficiency_{trig}')
    trigger_mc_efficiency[trig].SetDirectory(0)



final_spectra={}
final_spectra_ratio={}

current_trigger='JP2'
# load trigger efficiency from embedding
for trig in ['JP2', 'HT2']:
      # clone histogram for trigger efficiency
    final_spectra[trig] = normalized_events_livetime_prescale_maxnevents_1d[trig].Clone(f'{trig}_final')
    final_spectra[trig].SetDirectory(0)
    final_spectra[trig].SetTitle(f'{trig}')
    final_spectra[trig].SetLineColor( colors[trig])
    final_spectra[trig].SetMarkerColor(colors[trig])
    final_spectra[trig].SetMarkerStyle(markers[trig])
    final_spectra[trig].SetMarkerSize(1.2)
    final_spectra[trig]=divide_by_efficiency(final_spectra[trig], trigger_efficiency[trig])
    # scale by ratio_passed_events
    final_spectra[trig].Scale(ratio_passed_events[trig])

    final_spectra_ratio[trig] = mb_norm.Clone(f'{trig}_ratio')
    final_spectra_ratio[trig].SetLineColor( colors[trig])
    final_spectra_ratio[trig].SetMarkerColor(colors[trig])
    final_spectra_ratio[trig].SetMarkerStyle(markers[trig])
    final_spectra_ratio[trig].SetDirectory(0)
    final_spectra_ratio[trig].GetYaxis().SetTitle(f"Ratio MB / Trigger")
    final_spectra_ratio[trig].Divide(final_spectra[trig])
    

canvas.Divide(1, 2)
# Upper pad for histograms
pad1 = canvas.cd(1)
pad1.SetPad(0.0, 0.5, 1.0, 1.0)
pad1.SetTopMargin(0.1)
pad1.SetBottomMargin(0.0)
pad1.SetLogy()

# Set histogram styles
mb_norm.SetTitle("")
mb_norm.GetXaxis().SetLabelSize(0)
mb_norm.GetYaxis().SetTitleSize(0.06)

canvas.cd(1)
# Draw histograms
mb_norm.GetYaxis().SetRangeUser(0.7*10e-10,0.3* 10e-3)
mb_norm.Draw("E1")
final_spectra['JP2'].Draw("E1 SAME")
final_spectra['HT2'].Draw("E1 SAME")
mb_norm.Draw("E1 SAME")
mb_norm.GetXaxis().SetRangeUser(0, 42)

# Create legend
legend1 = ROOT.TLegend(0.7, 0.5, 0.9, 0.9)
legend1.AddEntry(mb_norm,  "MB ", "lep")
legend1.AddEntry(final_spectra['JP2'], 'JP2', "lep")
legend1.AddEntry(final_spectra['HT2'], 'HT2', "lep")
legend1.Draw()

# Lower pad for ratio
pad2 = canvas.cd(2)
pad2.SetPad(0.0, 0.0, 1.0, 0.5)
pad2.SetTopMargin(0.0)
pad2.SetBottomMargin(0.3)
pad2.SetGridy()

 # Set ratio histogram style

final_spectra_ratio['JP2'].SetTitle("")
final_spectra_ratio['JP2'].GetXaxis().SetTitle(" p_{t} [GeV/c]")
final_spectra_ratio['JP2'].GetXaxis().SetTitleSize(0.12)
final_spectra_ratio['JP2'].GetXaxis().SetLabelSize(0.10)


canvas.cd(2)
final_spectra_ratio['JP2'].GetXaxis().SetRangeUser(0, 42)
final_spectra_ratio['JP2'].GetYaxis().SetRangeUser(0, 3.1)
final_spectra_ratio['JP2'].Draw("E1")
final_spectra_ratio['HT2'].Draw("E1 SAME")


trigger_ratio=final_spectra_ratio['JP2'].Clone('trigger_ratio')
trigger_ratio.Divide(final_spectra_ratio['HT2'])
trigger_ratio.SetTitle("")
trigger_ratio.SetLineColor(ROOT.kBlack)
trigger_ratio.SetMarkerColor(ROOT.kBlack)
trigger_ratio.SetMarkerStyle(24)


legend2 = ROOT.TLegend(0.3, 0.66, 0.48, 0.97)
legend2.AddEntry(final_spectra_ratio['JP2'], 'MB/JP2', "lep")
legend2.AddEntry(final_spectra_ratio['HT2'], 'MB/HT2', "lep")
legend2.AddEntry(trigger_ratio, 'HT2/JP2', "lep")
legend2.Draw()

trigger_ratio.Draw("E1 SAME")   

# canvas.SaveAs(f'plots/ratio_MB_JP2_HT2_jet_yields.pdf')
